World Coins: A collection of coin images from 32 different currencies.

In [2]:
# Seznam knihoven
import os
from tensorflow.keras.applications import ResNet50, EfficientNetB3, MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as preprocess_mobilenet

In [3]:
# Napojení dat a vytvoření cesty
data_dir_train = "data/coins/data/train/"
data_dir_val = "data/coins/data/validation/"
log_dir = "models/training_logs/"
model_dir = "models/"

In [ ]:
# Parametry
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 100
NUM_CLASSES = 211
EPOCHS = 200

In [5]:
# Vytvoření složek, pokud neexistují
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

Změny augmentace

Každý model má jinou augmentaci kvůli odlišným vlastnostem architektury a způsobu předzpracování vstupních dat. Zde jsou hlavní důvody pro rozdílné nastavení.


ResNet50 - Pro velké modely a přesnost 🏋️‍♂️

ResNet50 -> Silná augmentace (odolnost proti deformacím, změny barev).


EfficientNetB3 - Nejlepší poměr výkon/výpočetní náročnost ⚖️

EfficientNetB3 -> Střední augmentace (jemné doladění, citlivost na detaily).


MobileNetV3 - Lehký model pro mobilní aplikace 

MobileNetV3 -> Nejjemnější augmentace (udržení rychlosti a stability).

Model ResNet50

In [ ]:
# ImageDataGenerator s odpovídajícím předzpracováním pro ResNet50
train_datagen_resnet = ImageDataGenerator(
    preprocessing_function=preprocess_resnet,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],  # Lehké změny jasu
    channel_shift_range=20.0) # Malé změny barev

val_datagen_resnet = ImageDataGenerator(preprocessing_function=preprocess_resnet)

# Načtení datasetu
train_generator_resnet = train_datagen_resnet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_resnet = val_datagen_resnet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_resnet)
validation_steps = max(len(val_generator_resnet), 1)

In [ ]:
# Načtení předtrénovaného modelu ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Přidání nových vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)  # Přidána hlubší Dense vrstva
x = BatchNormalization()(x)  # Normalizace pro stabilnější trénování
x = Dropout(0.4)(x)  # Silnější regulace
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# ResNet50 – ReLU
# ReLU (Rectified Linear Unit) je jednoduchá, rychlá a efektivní pro hluboké sítě, jako je ResNet.
# Výhoda: Dobře se hodí pro hlubší dense vrstvy, protože snižuje problém mizení gradientu.
# Doplněk: Použití BatchNormalization stabilizuje trénování a Dropout brání přeučení.

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, "best_ResNet50.keras"), save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
log_file = os.path.join(log_dir, "ResNet50.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])

In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:50]:
    layer.trainable = False

# Rekompilace
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Další fáze trénování
history_fine = model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])

# Uložení modelu
best_model_path = os.path.join(model_dir, "best_ResNet50.keras")
if os.path.exists(best_model_path):
    model.save(best_model_path)

Model EfficientNetB3

In [ ]:
train_datagen_efficientnet = ImageDataGenerator(
    preprocessing_function=preprocess_efficientnet,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.9, 1.1])  # Lehké změny jasu

val_datagen_efficientnet = ImageDataGenerator(preprocessing_function=preprocess_efficientnet)

train_generator_efficientnet = train_datagen_efficientnet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_efficientnet = val_datagen_efficientnet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_efficientnet)
validation_steps = max(len(val_generator_efficientnet), 1)

In [ ]:
# Načtení předtrénovaného modelu EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zmrazení základního modelu

# Přidání nových vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='swish')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# EfficientNetB3 – Swish
# Swish (varianty SiLU) funguje lépe než ReLU v optimalizovaných škálovaných sítích.
# Výhoda: Má hladší derivace, což umožňuje lepší průchod gradientu → efektivnější učení.
# Vhodnost: EfficientNet je škálovaný model, kde Swish zlepšuje přesnost při stejném počtu parametrů.

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, "best_EfficientNetB3.keras"), save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
log_file = os.path.join(log_dir, "EfficientNetB3.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_efficientnet,
    validation_data=val_generator_efficientnet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])


In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Rekompilace s nižší learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Další fáze trénování
history_fine = model.fit(
    train_generator_efficientnet,
    validation_data=val_generator_efficientnet,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])

# Uložení modelu
best_model_path = os.path.join(model_dir, "bbest_EfficientNetB3.keras")
if os.path.exists(best_model_path):
    model.save(best_model_path)


Model MobileNetV3

In [ ]:
train_datagen_mobilenet = ImageDataGenerator(
    preprocessing_function=preprocess_mobilenet,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.95, 1.05],  # Lehké změny jasu
    channel_shift_range=10.0) # Malé změny barev

val_datagen_mobilenet = ImageDataGenerator(preprocessing_function=preprocess_mobilenet)

train_generator_mobilenet = train_datagen_mobilenet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_mobilenet = val_datagen_mobilenet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_mobilenet)
validation_steps = max(len(val_generator_mobilenet), 1)

In [ ]:
# Načtení předtrénovaného modelu MobileNetV3
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zmrazení základního modelu

# Přidání vlastních vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='hard_swish')(x)
x = Dropout(0.4)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# MobileNetV3 – Hard-Swish
# Hard-Swish je zjednodušená verze Swish, navržená pro mobilní zařízení.
# Výhoda: Je rychlejší než Swish, ale poskytuje podobný výkon. Šetří výpočetní zdroje při zachování efektivity.
# Vhodnost: MobileNetV3 je optimalizovaný pro rychlost a nízkou spotřebu.

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, "best_MobileNetV3.keras"), save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
log_file = os.path.join(log_dir, "MobileNetV3.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_mobilenet,
    validation_data=val_generator_mobilenet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])

In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:200]:
    layer.trainable = False

# Rekompilace s nižší learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Další fáze trénování
history_fine = model.fit(
    train_generator_mobilenet,
    validation_data=val_generator_mobilenet,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger])

# Uložení modelu
best_model_path = os.path.join(model_dir, "best_MobileNetV3.keras")
if os.path.exists(best_model_path):
    model.save(best_model_path)